# Detección de Grietas en Superficies de Concreto usando Redes Neuronales (FC vs. CNN)

## Introducción

En esta asignación se aborda el problema de detección de grietas en superficies de concreto utilizando imágenes RGB provenientes del dataset **Surface Crack Detection** disponible en Kaggle. Este conjunto contiene un total de 40,000 imágenes de 227x227 píxeles, balanceado entre clases positivas (grietas) y negativas (sin grietas).

El objetivo principal es desarrollar, entrenar y comparar el desempeño de dos enfoques de redes neuronales:

- Una red neuronal **totalmente conectada (Fully Connected - FC)**, que recibe imágenes aplanadas como vectores de entrada.
- Una red **convolucional (CNN)**, diseñada específicamente para tareas de visión por computadora, capaz de aprender representaciones espaciales jerárquicas.

Ambos modelos serán evaluados en la tarea de **clasificación binaria**, determinando si una imagen contiene o no una grieta.

## Objetivos

- Cargar y explorar el dataset Surface Crack Detection.
- Preprocesar las imágenes, incluyendo normalización y partición en conjuntos de entrenamiento, validación y prueba.
- Diseñar e implementar una red totalmente conectada que reciba imágenes aplanadas.
- Diseñar e implementar una red convolucional adaptada a la estructura espacial de las imágenes.
- Entrenar ambos modelos y graficar la evolución de la función de pérdida y la precisión durante el entrenamiento.
- Evaluar el desempeño de cada red en el conjunto de prueba utilizando métricas como **accuracy**, **precision**, **recall** y **F1-score**.
- Comparar y analizar los resultados obtenidos, identificando ventajas y limitaciones de cada arquitectura para esta tarea.

## Importancia del problema

La detección automatizada de grietas es fundamental para el mantenimiento preventivo de infraestructuras como puentes, carreteras y edificios. Métodos basados en visión por computadora ofrecen una alternativa rápida, económica y no invasiva frente a inspecciones manuales. Evaluar diferentes arquitecturas de redes neuronales en esta tarea permite identificar soluciones eficientes que puedan escalar a sistemas reales de monitoreo estructural.



---

## Descripción del Dataset

El dataset utilizado en este proyecto es el **Surface Crack Detection Dataset**, disponible públicamente en [Kaggle](https://www.kaggle.com/datasets/arunrk7/surface-crack-detection). Este conjunto de datos fue diseñado para entrenar modelos de clasificación binaria que determinen la presencia o ausencia de grietas en superficies de concreto.

### Características del Dataset

- **Total de imágenes:** 40,000  
- **Clases:**  
  - `Positive` (1): imagen contiene una grieta  
  - `Negative` (0): imagen sin grietas  
- **Tamaño de las imágenes:** 227×227 píxeles  
- **Canales:** 3 (RGB)  
- **Formato:** JPG  
- **Distribución balanceada:** 20,000 imágenes por clase

### Estructura del Dataset

El dataset está organizado en dos carpetas principales:

```
Surface_Crack_Detection/
├── Positive/   # Imágenes con grietas (20,000)
└── Negative/   # Imágenes sin grietas (20,000)
```



### Origen de los datos

Las imágenes fueron generadas a partir de 458 imágenes reales de superficies de concreto, recolectadas en diferentes condiciones de iluminación y textura. Posteriormente se aplicaron técnicas de segmentación y recorte para crear imágenes individuales de alta resolución. El dataset fue diseñado para facilitar tareas de clasificación, detección y segmentación de defectos estructurales.

### Consideraciones

- El tamaño uniforme de las imágenes (227x227) facilita su uso directo en redes convolucionales sin necesidad de redimensionamiento adicional.
- Aunque el dataset incluye variabilidad en texturas y condiciones visuales, no contiene etiquetas de localización (como bounding boxes o máscaras), por lo que se usa principalmente para **clasificación binaria**.



---

### Paso 1: Cargar y explorar el dataset

- Descargar y descomprimir el dataset desde Kaggle.
- Verificar la estructura de carpetas:

```
Surface_Crack_Detection/
├── Positive/ # Imágenes con grietas
└── Negative/ # Imágenes sin grietas
```

- Contar el número total de imágenes por clase.
- Visualizar algunas muestras de ambas clases para entender el tipo de imágenes.

In [ ]:
# Tu codigo aquí

---
### Paso 2: Preprocesamiento y partición del dataset

- Asignar etiqueta `1` a las imágenes de `Positive` y `0` a las de `Negative`.
- Cargar las imágenes con tamaño fijo (227×227), redimensionarlas y convertirlas a tensores (PyTorch) o arrays (TensorFlow).
- Normalizar los valores de píxeles al rango [0, 1].

#### Esquema de partición:

1. Reservar **20 % del total** como conjunto de prueba final (`test`), que **no se toca durante el entrenamiento ni validación**.
2. Del 80 % restante (conjunto de desarrollo), separar:
   - **80 % para entrenamiento** → 64 % del total
   - **20 % para validación** → 16 % del total

#### Resumen de proporciones:

| Conjunto     | Porcentaje del total | Uso                                 |
|--------------|----------------------|--------------------------------------|
| Entrenamiento| 64 %                 | Entrenar los modelos                 |
| Validación   | 16 %                 | Ajustar hiperparámetros y arquitectura |
| Prueba       | 20 %                 | Evaluación final del rendimiento     |

#### Consideraciones adicionales:

- La división debe ser **estratificada** para conservar el balance entre clases en cada subconjunto.
- Aplicar **data augmentation solo al conjunto de entrenamiento**:
  - Rotaciones aleatorias
  - Flips horizontales
  - Zoom y desplazamientos leves
- Validar que no haya **fugas de datos** entre los subconjuntos (por ejemplo, imágenes duplicadas en entrenamiento y prueba).


In [ ]:
# Tu codigo aquí

---
### Paso 3: Definir la red neuronal totalmente conectada (FC)

En este paso se implementa una red neuronal completamente conectada (Fully Connected - FC), la cual recibe imágenes aplanadas como vectores unidimensionales y predice si la imagen contiene una grieta o no.

### Arquitectura propuesta: Red neuronal totalmente conectada (FC) con 2 salidas (clasificación multiclase)

#### Entrada
- Dimensión de entrada: **3 × 227 × 227 = 154,587**
- Las imágenes RGB se aplanan en un vector para ser procesadas por capas lineales

#### Estructura de la red
1. **Capa Lineal 1:** 154,587 → 1024
2. **Función de activación:** ReLU
3. **Capa Lineal 2:** 1024 → 512
4. **Función de activación:** ReLU
5. **Capa Lineal 3:** 512 → 128
6. **Función de activación:** ReLU
7. **Capa Lineal de salida:** 128 → 2  
   - Cada unidad de salida representa una clase:
     - `Clase 0`: sin grieta
     - `Clase 1`: con grieta

#### Función de pérdida
- **`CrossEntropyLoss`**  
  - Esta función aplica `softmax` internamente
  - Las etiquetas deben ser enteros (`0` o `1`), **no one-hot encoded**

#### Optimizador
- **Adam**
  - Tasa de aprendizaje: **0.001**
  - Parámetros por defecto: `betas=(0.9, 0.999)`, `eps=1e-08`

#### Configuración de entrenamiento
- **Número de épocas:** 25
- **Tamaño de batch:** 32
- **Métricas de evaluación:** Accuracy, Precision, Recall, F1-score (macro y por clase), ROC-AUC, Tiempo de entrenamiento, Tiempo de Inferencia, Gráfico de Training Loss and Validation Loss vs Epochs.

#### Notas
- Al usar dos salidas y `CrossEntropyLoss`, se está formalizando el problema como clasificación multiclase, aunque haya solo dos clases.
- Esto permite extender fácilmente a problemas con más categorías en el futuro si se desea.



In [ ]:
# Tu codigo aquí

---
### Paso 4: Definir la red neuronal convolucional (CNN)

En este paso se implementa una red neuronal convolucional (Convolutional Neural Network - CNN), diseñada específicamente para capturar patrones espaciales en imágenes, como bordes, texturas y formas locales. A diferencia de la red FC, esta arquitectura mantiene la estructura espacial de las imágenes RGB y aprovecha las propiedades de las convoluciones para mejorar el aprendizaje.

### Arquitectura propuesta: CNN con 2 salidas (clasificación multiclase)

#### Entrada
- Dimensión de entrada: **3 × 227 × 227**
- Las imágenes RGB se procesan conservando su estructura espacial y canal de color

#### Estructura de la red
1. **Capa Convolucional 1:** 3 canales → 32 filtros, tamaño de kernel 3×3, padding 1
2. **Función de activación:** ReLU
3. **MaxPooling:** 2×2
4. **Capa Convolucional 2:** 32 → 64 filtros, tamaño 3×3, padding 1
5. **Función de activación:** ReLU
6. **MaxPooling:** 2×2
7. **Capa Convolucional 3:** 64 → 128 filtros, tamaño 3×3, padding 1
8. **Función de activación:** ReLU
9. **MaxPooling:** 2×2
10. **Flatten:** la salida convolucional se convierte en un vector
11. **Capa Lineal 1:** salida → 128 unidades
12. **Función de activación:** ReLU
13. **Capa Lineal de salida:** 128 → 2  
    - Cada unidad representa una clase:
      - `Clase 0`: sin grieta  
      - `Clase 1`: con grieta

#### Función de pérdida
- **`CrossEntropyLoss`**
  - Incluye la activación `softmax` internamente
  - Las etiquetas deben estar codificadas como enteros (`0` o `1`), no en formato one-hot

#### Optimizador
- **Adam**
  - Tasa de aprendizaje: **0.001**
  - Parámetros por defecto: `betas=(0.9, 0.999)`, `eps=1e-08`

#### Configuración de entrenamiento
- **Número de épocas:** 25
- **Tamaño de batch:** 32
- **Métricas de evaluación:** Accuracy, Precision, Recall, F1-score (macro y por clase), ROC-AUC, Tiempo de entrenamiento, Tiempo de Inferencia, Gráfico de Training Loss and Validation Loss vs Epochs. 

#### Notas
- La CNN es capaz de extraer características visuales jerárquicas directamente de las imágenes, lo cual suele mejorar la capacidad de generalización.
- Esta red puede extenderse fácilmente agregando más capas convolucionales, dropout o batch normalization si se desea mejorar el rendimiento o controlar el sobreajuste.


In [ ]:
# Tu codigo aquí

---
### Paso 5: Tabla resumen de resultados por arquitectura

Una vez entrenados y evaluados ambos modelos (la red totalmente conectada y la red convolucional), construye una **tabla comparativa de resultados** que resuma el desempeño de cada arquitectura en el conjunto de prueba.

#### Métricas a reportar:

- **Accuracy**
- **Precision (macro y por clase)**
- **Recall (macro y por clase)**
- **F1-score (macro y por clase)**
- **ROC-AUC**
- **Tiempo total de entrenamiento** (en segundos)
- **Tiempo promedio de inferencia por imagen** (en milisegundos)
- **Gráfico de Training Loss y Validation Loss vs. Epochs**

#### Formato sugerido:

| Modelo         | Accuracy | Precision (macro) | Recall (macro) | F1-score (macro) | ROC-AUC | Entrenamiento (s) | Inferencia (ms) |
|----------------|----------|-------------------|----------------|------------------|---------|--------------------|------------------|
| FC             | 0.XXX    | 0.XXX             | 0.XXX          | 0.XXX            | 0.XXX   | XXX                | X.XX             |
| CNN            | 0.XXX    | 0.XXX             | 0.XXX          | 0.XXX            | 0.XXX   | XXX                | X.XX             |

> Acompaña esta tabla con un gráfico que muestre la evolución de la pérdida durante las 25 épocas de entrenamiento:
> - Training Loss vs Epoch
> - Validation Loss vs Epoch

---



### Paso 6: Preguntas de análisis

Con base en los resultados obtenidos, responde de forma clara y justificada las siguientes preguntas:

1. **¿Cuál modelo obtuvo mejor rendimiento general?**  
   Considera todas las métricas relevantes y argumenta tu respuesta.

2. **¿Qué diferencias observaste entre el comportamiento de la red FC y la CNN en cuanto a overfitting o estabilidad durante el entrenamiento?**

3. **¿El tiempo de entrenamiento y la inferencia justifican el uso de una arquitectura más compleja como la CNN en este problema?**

4. **¿Qué métrica consideras más importante en este contexto y por qué?**  
   Por ejemplo, en una aplicación real, ¿te preocupa más evitar falsos negativos o falsos positivos?

5. **Si tuvieras que mejorar aún más el modelo CNN, ¿qué cambios introducirías en la arquitectura o en el proceso de entrenamiento y por qué?**  
   (regularización, dropout, batch normalization, más datos, fine-tuning con modelos preentrenados, etc.)

---

## Rúbrica de evaluación del proyecto

El proyecto se compone de seis pasos estructurados. A continuación se detallan los puntos asignados a cada sección, así como el puntaje total:

| Sección                                                                 | Puntos |
|------------------------------------------------------------------------|--------|
| **Paso 1:** Cargar y explorar el dataset                               | 5      |
| **Paso 2:** Preprocesamiento y partición del dataset                   | 5      |
| **Paso 3:** Red neuronal totalmente conectada (FC)                     | 60     |
| **Paso 4:** Red neuronal convolucional (CNN)                           | 60     |
| **Paso 5:** Tabla resumen de resultados y visualización de métricas   | 10     |
| **Paso 6:** Preguntas de análisis y reflexión                          | 20     |
| **Total**                                                              | **160** |

---

**Nota:** Para obtener la máxima puntuación se requiere justificar adecuadamente cada decisión de diseño, mantener buena organización en el notebook, utilizar visualizaciones apropiadas, y presentar resultados bien interpretados.

---
